<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/Kopi_av_Co2_recompression2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install neqsim
#https://reaktoro.org/installation/installation-in-google-colab.html
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

In [18]:
import pandas as pd
#from neqsim import jneqsim as neqsim
from neqsim.thermo import fluid_df, TPflash, printFrame
import reaktoro as rkt
import json
from neqsim import jneqsim as neqsim
from reaktoro import *

# Create NeqSim unit operations using Reaktoro

## Gas phase Reaction NeqSim unit opeation

In [14]:
from neqsim.process.unitop import unitop
from jpype import JImplements, JOverride

class GasReactor(unitop):
    def __init__(self):
        super().__init__()
        self.serialVersionUID = None
        self.name = None
        self.inletstream = None
        self.outletstream = None

    def setInletStream(self, stream):
        self.inletstream = stream
        self.outletstream = stream.clone()

    def getOutletStream(self):
        return self.outletstream

    def calc_reaktoro(self):
      db = NasaDatabase("nasa-cea")
      gases = GaseousPhase("CO2 H2O H2S SO2 NO2 NO O2 H2 CO CH4 N2 H2SO4 CH3COOH N2O HNO3 NO3 N2O5 NH3 HNO3 S8")
      system = ChemicalSystem(db, gases)

      state = ChemicalState(system)
      state.temperature(self.inletstream.getTemperature("C"), "celsius")
      state.pressure(self.inletstream.getPressure("bara"), "bar")
      state.set("CO2", self.inletstream.getFluid().getComponent("CO2").getNumberOfmoles(), "mol")
      state.set("N2", self.inletstream.getFluid().getComponent("nitrogen").getNumberOfmoles(), "mol")
      state.set("O2", self.inletstream.getFluid().getComponent("oxygen").getNumberOfmoles(), "mol")
      state.set("H2O", self.inletstream.getFluid().getComponent("water").getNumberOfmoles(), "mol")
      state.set("H2S", self.inletstream.getFluid().getComponent("H2S").getNumberOfmoles(), "mol")
      state.set("NO2",  self.inletstream.getFluid().getComponent("NO2").getNumberOfmoles(), "mol")
      state.set("SO2",  self.inletstream.getFluid().getComponent("SO2").getNumberOfmoles(), "mol")
      state.set("NH3",  self.inletstream.getFluid().getComponent("ammonia").getNumberOfmoles(), "mol")
      #print("=== INITIAL STATE ===")
      #print(state)
      solver = EquilibriumSolver(system)
      solver.solve(state)

      CO2_concentration = state.speciesAmount("CO2")
      N2_concentration = state.speciesAmount("N2")
      O2_concentration = state.speciesAmount("O2")
      NO2_concentration = state.speciesAmount("NO2")
      CO_concentration = state.speciesAmount("CO")
      SO2_concentration = state.speciesAmount("SO2")
      H2S_concentration = state.speciesAmount("H2S")
      NH3_concentration = state.speciesAmount("NH3")
      H2SO4_concentration = state.speciesAmount("H2SO4")
      HNO3_concentration = state.speciesAmount("HNO3")
      H2O_concentration = state.speciesAmount("H2O")

      molarrates = [CO2_concentration, N2_concentration, O2_concentration, NO2_concentration, CO_concentration, SO2_concentration, H2S_concentration, NH3_concentration, H2SO4_concentration, HNO3_concentration, H2O_concentration]
      self.outletstream.getFluid().setMolarFlowRates(molarrates)
      self.outletstream.run()

    @JOverride
    def run(self):
      #self.serialVersionUID = uuid
      print("ReaktoroUnitOperation.run()")
      self.calc_reaktoro()

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name
      }
      return json.dumps(data_dict)


## Liquid phase reaction NeqSim unit operation

In [22]:
class LiquidReactor(unitop):
    def __init__(self):
        super().__init__()
        self.serialVersionUID = None
        self.name = ""
        self.sullfuricacid_weight_percent = None
        self.nitric_acid_weight_percent = None
        self.inletstream = None
        self.outletstream = None

    def setInletStream(self, stream):
        self.inletstream = stream
        self.outletstream = stream.clone()

    def getOutletStream(self):
        return self.outletstream

    def getOutletStream(self):
        return self.outletstream

    def calc_reaktoro(self):
      db = PhreeqcDatabase("phreeqc.dat")

      solution = AqueousPhase(speciate("H O C N S"))
      solution.set(ActivityModelPhreeqc(db))

      gases = GaseousPhase("N2(g) CO2(g) H2O(g) H2S(g) O2(g) NH3(g) CH4(g)")
      gases.set(ActivityModelPengRobinsonPhreeqcOriginal())

      minerals = MineralPhases()

      system = ChemicalSystem(db, solution, gases, minerals)

      state = ChemicalState(system)
      state.temperature(self.inletstream.getTemperature("C"), "celsius")
      state.pressure(self.inletstream.getPressure("atm"), "atm")
      state.add("H2O"    , self.inletstream.getFluid().getComponent("water").getFlowRate("kg/sec"), "kg")
      state.add("CO2"    , self.inletstream.getFluid().getComponent("CO2").getFlowRate("mole/sec"), "mol")
      state.add("N2"    , self.inletstream.getFluid().getComponent("nitrogen").getFlowRate("mole/sec"), "mol")
      state.add("O2"    , self.inletstream.getFluid().getComponent("oxygen").getFlowRate("mole/sec"), "mol")
      state.add("H2S"    , self.inletstream.getFluid().getComponent("H2S").getFlowRate("mole/sec"), "mol")

      #H2SO4
      state.add("SO4-2"    , self.inletstream.getFluid().getComponent("sulfuric acid").getFlowRate("mole/sec"), "mol")
      state.set("H+"    , 2*self.inletstream.getFluid().getComponent("sulfuric acid").getFlowRate("mole/sec"), "mol")

      #HNO3
      state.add("NO3-"    , self.inletstream.getFluid().getComponent("nitric acid").getFlowRate("mole/sec"), "mol")
      state.add("H+"    , self.inletstream.getFluid().getComponent("nitric acid").getFlowRate("mole/sec"), "mol")

      print("=== START STATE ===")
      print(state)
      equilibrate(state)

      aprops = AqueousProps(state)


      print(aprops)

      solver = EquilibriumSolver(system)
      solver.solve(state)
      print("=== FINAL STATE ===")
      print(state)

      aprops = AqueousProps(state)

      CO2_concentration = state.speciesAmount("CO2(g)")
      N2_concentration = state.speciesAmount("N2(g)")
      O2_concentration = state.speciesAmount("O2(g)")
      H2S_concentration = state.speciesAmount("H2S(g)")
      NH3_concentration = state.speciesAmount("NH3(g)")
      H2O_concentration = state.speciesAmount("H2O(g)")

      molarrates = [CO2_concentration, N2_concentration, O2_concentration, 0, 0, 0, H2S_concentration, NH3_concentration, 0, 0, H2O_concentration]
      self.outletstream.getFluid().setMolarFlowRates(molarrates)
      self.outletstream.run()

      molecular_weight_sulfuric_acid = 98.079
      self.sullfuricacid_weight_percent = state.speciesAmount("SO4-2")*molecular_weight_sulfuric_acid/(state.speciesAmount("H2O")*18.02/100.0)*100
      print('sulfuric acid wt% ', self.sullfuricacid_weight_percent)
      sullfuricacid_molality = state.speciesAmount("SO4-2")/(state.speciesAmount("H2O")*18.02/100.0)
      print('sullfuricacid_molality ', sullfuricacid_molality)
      self.nitric_acid_weight_percent = state.speciesAmount("NO3-")*molecular_weight_sulfuric_acid/(state.speciesAmount("H2O")*18.02/100.0)*100

      print(aprops)

    @JOverride
    def run(self):
      #self.serialVersionUID = uuid
      print("ReaktoroUnitOperation.run()")
      self.calc_reaktoro()

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name,
            "sulfuric_acid": self.sullfuricacid_weight_percent,
            "nitric_acid": self.nitric_acid_weight_percent,
      }
      return json.dumps(data_dict)

# Feed fluid definition

In [16]:
# Create a gas-condensate fluid
feedfluid = {'ComponentName':  ["CO2", "nitrogen", "oxygen", "NO2", "CO", "SO2", "H2S", "ammonia", "sulfuric acid", "nitric acid", "water"],
        'MolarComposition[-]':  [1.0, 100.0e-6, 30e-6, 5e-6, 5e-6, 1e-6, 5e-6, 1e-6, 0.0, 0.0, 0.1]
}

feedfluiddf = pd.DataFrame(feedfluid)
print("Natural Gas Fluid:\n")
print(feedfluiddf.head(30).to_string())
feedfluid_neqsim = fluid_df(feedfluiddf,modelName='cpa')
feedfluid_neqsim.setMixingRule(10)

feedfluid_neqsim.setTemperature(30.0, 'C')
feedfluid_neqsim.setPressure(1.2, 'bara')
TPflash(feedfluid_neqsim)
printFrame(feedfluid_neqsim)

Natural Gas Fluid:

    ComponentName  MolarComposition[-]
0             CO2             1.000000
1        nitrogen             0.000100
2          oxygen             0.000030
3             NO2             0.000005
4              CO             0.000005
5             SO2             0.000001
6             H2S             0.000005
7         ammonia             0.000001
8   sulfuric acid             0.000000
9     nitric acid             0.000000
10          water             0.100000
| 0                    | 1          | 2                    | 3                    | 4   | 5   | 6               |
|:---------------------|:-----------|:---------------------|:---------------------|:----|:----|:----------------|
|                      | total      | GAS                  | AQUEOUS              |     |     |                 |
| CO2                  | 9.08969E-1 | 9.63912E-1           | 5.1222E-4            |     |     | [mole fraction] |
| nitrogen             | 9.08969E-5 | 9.63942E-5        

# Recompression process



In [19]:
from neqsim.thermo import fluid, TPflash, phaseenvelope, fluid_df
from neqsim.process import stream, compressor, cooler, separator, mixer, valve, pump, clearProcess

clearProcess()
# Create feed stream
feedStream = stream('feed1 ', feedfluid_neqsim)
feedStream.setTemperature(30.0, "C")
feedStream.setPressure(1.2, "bara")
feedStream.setFlowRate(10.0, "kg/sec")
feedStream.run()

gasreacot1 = GasReactor()
gasreacot1.setName('gas reactor 1')
gasreacot1.setInletStream(feedStream)
gasreacot1.run()

# Create compressor
compressor1 = compressor(' compressor 1', gasreacot1.getOutletStream())
compressor1.setOutletPressure(5.0, "bara")
compressor1.setIsentropicEfficiency(0.8)
compressor1.run()

# Create cooler
cooler1 = cooler('cooler 1', compressor1.getOutStream())
cooler1.setOutTemperature(30.0, "C")
cooler1.run()


liqreactor = LiquidReactor()
liqreactor.setName('liquid reactor 1')
liqreactor.setInletStream(cooler1.getOutStream())
liqreactor.run()

# Create separator
separator1 = separator('sep 1', liqreactor.getOutletStream())

co2_process = neqsim.process.processmodel.ProcessSystem()
co2_process.add(feedStream)
co2_process.add(gasreacot1)
co2_process.add(compressor1)
co2_process.add(cooler1)
co2_process.add(liqreactor)
co2_process.add(separator1)
co2_process.run()

printFrame(cooler1.getOutletStream().getFluid())

printFrame(liqreactor.getOutletStream().getFluid())

ReaktoroUnitOperation.run()
ReaktoroUnitOperation.run()
=== START STATE ===
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |    303.1500 |    K |
| Pressure        |      5.0000 |  bar |
| Charge:         | -1.1999e-15 |  mol |
| Element Amount: |             |      |
| :: H            |  3.9291e+00 |  mol |
| :: C            |  2.1822e+02 |  mol |
| :: N            |  4.4962e-02 |  mol |
| :: O            |  4.3842e+02 |  mol |
| :: S            |  1.3051e-03 |  mol |
| Species Amount: |             |      |
| :: CO3-2        |  1.0000e-16 |  mol |
| :: H+           |  2.6103e-03 |  mol |
| :: H2O          |  1.9632e+00 |  mol |
| :: CO2          |  2.1822e+02 |  mol |
| :: (CO2)2       |  1.0000e-16 |  mol |
| :: SO4-2        |  1.3051e-03 |  mol |
| :: HS-          |  1.0000e-16 |  mol |
| :: H2S          |  1.9994e-16 |  mol |
| :: (H2S)2       |  1.0000e-16 |  mol |
| :: HSO4-        |  1

In [20]:
json_report= str(neqsim.process.util.report.Report(co2_process).generateJsonReport())
output = json.loads(json_report)
print(output['liquid reactor 1'])

TypeError: Object of type real1st is not JSON serializable